# Import Packages

In [1]:
import pandas as pd
import glob as glob
import numpy as np
import geopandas as ge
import matplotlib.pyplot as plt
import seaborn as sns

# Import Dataset

In [31]:
df_jams = pd.read_csv("../data/raw/aggregate_median_jams_Kota Bogor.csv")
df_alerts = pd.read_csv("../data/raw/aggregate_alerts_Kota Bogor.csv")
df_irre = pd.read_csv("../data/raw/aggregate_median_irregularities_Kota Bogor.csv")

df_jams.drop("Unnamed: 0", axis=1, inplace=True)

## Checking data on Alerts dataset & Info

In [19]:
df_jams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102322 entries, 0 to 102321
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   time                       102322 non-null  object 
 1   kemendagri_kabupaten_kode  102322 non-null  float64
 2   kemendagri_kabupaten_nama  102322 non-null  object 
 3   street                     100529 non-null  object 
 4   level                      102322 non-null  int64  
 5   median_length              102322 non-null  float64
 6   median_delay               102322 non-null  float64
 7   median_speed_kmh           102322 non-null  float64
 8   total_records              102322 non-null  int64  
 9   id                         102322 non-null  int64  
 10  date                       102322 non-null  object 
 11  median_level               102322 non-null  float64
 12  geometry                   102322 non-null  object 
dtypes: float64(5), int64(3), obje

In [20]:
df_alerts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         9762 non-null   int64  
 1   time                       9762 non-null   object 
 2   kemendagri_kabupaten_kode  9762 non-null   float64
 3   kemendagri_kabupaten_nama  9762 non-null   object 
 4   street                     8827 non-null   object 
 5   type                       9762 non-null   object 
 6   avg_location               9762 non-null   object 
 7   total_records              9762 non-null   int64  
 8   date                       9762 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 686.5+ KB


In [21]:
df_jams.sample(10)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
86342,2022-08-27 12:00:00.000,32.71,KOTA BOGOR,Tumenggung Wiradireja,3,378.0,111.0,8.165,2,36710961,2022-08-27,3.0,"LINESTRING (106.830791 -6.58957, 106.830872 -6..."
16193,2022-07-16 14:00:00.000,32.71,KOTA BOGOR,Siliwangi,3,956.0,327.0,7.155,56,34104928,2022-07-16,3.0,"MULTILINESTRING ((106.808862 -6.614809, 106.80..."
93477,2022-08-31 08:00:00.000,32.71,KOTA BOGOR,NaN,3,182.0,106.0,4.780,2,36956547,2022-08-31,3.0,"LINESTRING (106.847234 -6.655619, 106.847274 -..."
88534,2022-08-28 12:00:00.000,32.71,KOTA BOGOR,RE Martadinata,3,1365.0,165.0,13.310,7,36787091,2022-08-28,3.0,"MULTILINESTRING ((106.79016 -6.582906, 106.787..."
339,2022-07-06 10:00:00.000,32.71,KOTA BOGOR,Pangeran Asogiri,2,602.0,117.0,9.395,6,33469357,2022-07-06,2.0,"MULTILINESTRING ((106.823206 -6.557153, 106.82..."
100511,2022-09-04 07:00:00.000,32.71,KOTA BOGOR,Surya Kencana,2,827.0,144.0,12.530,9,37224250,2022-09-04,2.0,"MULTILINESTRING ((106.799491 -6.604398, 106.80..."
7341,2022-07-11 07:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,1,1594.0,86.0,27.840,43,33770668,2022-07-11,1.0,"MULTILINESTRING ((106.818424 -6.62308, 106.819..."
7424,2022-07-11 08:00:00.000,32.71,KOTA BOGOR,Perintis Kemerdekaan,2,499.0,93.0,11.190,13,33770729,2022-07-11,2.0,"MULTILINESTRING ((106.787584 -6.595391, 106.78..."
69708,2022-08-16 13:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Tajur,1,1547.0,79.0,27.620,25,36089682,2022-08-16,1.0,"MULTILINESTRING ((106.816527 -6.621295, 106.81..."
80180,2022-08-23 15:00:00.000,32.71,KOTA BOGOR,Kedunghalang Raya,1,1448.0,63.5,21.840,26,36473522,2022-08-23,1.0,"MULTILINESTRING ((106.805846 -6.545258, 106.80..."


# Data Quality Checking & Feature Transformation

## Jams Data

In [22]:
df_jams.isna().sum()

time                            0
kemendagri_kabupaten_kode       0
kemendagri_kabupaten_nama       0
street                       1793
level                           0
median_length                   0
median_delay                    0
median_speed_kmh                0
total_records                   0
id                              0
date                            0
median_level                    0
geometry                        0
dtype: int64

In [23]:
df_jams.dropna(subset=['street'], axis=0, inplace=True)

df_jams.shape

(100529, 13)

## Alerts Data

In [35]:
df_alerts.isna().sum()

id                             0
time                           0
kemendagri_kabupaten_kode      0
kemendagri_kabupaten_nama      0
street                       935
type                           0
avg_location                   0
total_records                  0
date                           0
dtype: int64

In [25]:
df_alerts.dropna(subset=["street"], axis=0,inplace=True)

df_alerts.shape

(8827, 9)

## Transforming Data